-------------- Real Project Time -------------

Use tensorflow.keras library for models. And you are free to use chatgpt here at this stage - Yayyyyyyyy\
Use it where you need help, not to complete the assignment lol

In [1]:
#import libraries
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

Define a function to extract features from the given filename. You will have to use librosa library for this. You can vary the features however you like. The preferred features are mfcc, chroma and melspectogram.

In [250]:
def extract_features(filename):
    y, sr = librosa.load(filename)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    return np.hstack([np.mean(mfcc, axis=1), np.mean(chroma, axis=1), np.mean(mel, axis=1), np.mean(contrast, axis=1)])
    # pass

Now, load the data from the database.

You can find the database from https://zenodo.org/records/1188976#.XsAXemgzaUk

You need to only download the audio files and not the video files. Also, read up on the naming convention of the files from the webpage.

In [251]:
def load_data(test_size):
    directories = [os.path.join(os.getcwd(), "Audio files Song"), os.path.join(os.getcwd(), "Audio files Speech")]
    X = []
    y = []
    for directory in directories:
        subdirectories = [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
        for subdirectory in subdirectories:
            files = [os.path.join(directory, subdirectory, name) for name in os.listdir(os.path.join(directory, subdirectory))]
            for file in files:
                features = extract_features(file)
                filename = os.path.basename(file)
                emotion_code = int(filename.split("-")[2])
                emotion_one_hot = np.zeros(8)
                emotion_one_hot[emotion_code - 1] = 1      
                X.append(features)
                y.append(emotion_one_hot)
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    print(X_train.shape)
    return X_train, X_test, y_train, y_test
    # pass

Use the load_data function to load data and split the data using the train_test_split function.

In [252]:
#split the dataset into training and testing data with testing data = 0.2 of total data
X_train, X_test, y_train, y_test = load_data(0.2)

(1961, 197)


Now you need to define the model for this computation.
For this, we will use simple neural network for this.\
The model will have a hidden layer of 300 nodes and an output layer of [number of emotions] nodes.\
You have to use relu as activation function for the hidden layer and softmax for the output layer.\
Keep the loss categorical-crossentropy, optimizer as adam and accuracy as metric.\
You can use batch-size as 256 and epochs as 300, but it is left upto you :)

In [254]:
#define the model.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam
from  keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

model = Sequential()
model.add(Dense(300, input_shape=(X_train.shape[1],), activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(8, activation='softmax'))
# optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# introducing callbacks
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_accuracy', save_best_only=True)
# early_stopping = EarlyStopping(monitor='val_accuracy', patience=100)
def scheduler(epoch, lr):
    if epoch < 600:
        return lr
    else:
        return lr * 0.8
learning_rate_scheduler = LearningRateScheduler(scheduler)
callbacks = [checkpoint, learning_rate_scheduler]


c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Now everything is easy-peezy. Just fit the model on training data as you usually do, predict for the testing data, print the accuracy. Hehe

In [255]:
#fit the model on training data with validation split of 0.2
model.fit(X_train, y_train, validation_split = 0.2 , epochs=1200, batch_size=256, callbacks=callbacks)

Epoch 1/1200
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.1604 - loss: 2.1402 - val_accuracy: 0.1756 - val_loss: 2.0402 - learning_rate: 0.0010
Epoch 2/1200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1871 - loss: 2.0383 - val_accuracy: 0.1756 - val_loss: 1.9697 - learning_rate: 0.0010
Epoch 3/1200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2041 - loss: 1.9885 - val_accuracy: 0.2468 - val_loss: 1.9405 - learning_rate: 0.0010
Epoch 4/1200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2419 - loss: 1.9390 - val_accuracy: 0.2901 - val_loss: 1.8736 - learning_rate: 0.0010
Epoch 5/1200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2905 - loss: 1.8950 - val_accuracy: 0.2952 - val_loss: 1.8460 - learning_rate: 0.0010
Epoch 6/1200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3004 - loss: 1.8298 - val_accuracy: 0.3435 - val_loss: 1.7920 - learning_rate: 0.0010
Epoch 7/1200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2794 - loss: 1.8081 - val_acc

In [256]:
#predict for testing data and printout the accuracy
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis = 1), np.argmax(y_pred, axis = 1))
print("Accuracy:", accuracy)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Accuracy: 0.7474541751527495


Now comes the learning part :)\
Here's a new thing. Print the classification report of your predictions.

In [257]:
#print the classification report for the model
from sklearn.metrics import classification_report
report = classification_report(np.argmax(y_test, axis = 1), np.argmax(y_pred, axis = 1))
print(report)

              precision    recall  f1-score   support

           0       0.69      0.81      0.74        43
           1       0.87      0.84      0.86        77
           2       0.77      0.75      0.76        72
           3       0.70      0.68      0.69        73
           4       0.80      0.86      0.83        77
           5       0.78      0.70      0.74        83
           6       0.48      0.50      0.49        32
           7       0.66      0.68      0.67        34

    accuracy                           0.75       491
   macro avg       0.72      0.73      0.72       491
weighted avg       0.75      0.75      0.75       491



Just for fun. You can explore on callbacks used in model.fit().\
For resources, you can go to https://keras.io/api/callbacks/ and explore on your own. Or just chatgpt :)
Now try the model using EarlyStopping callback, LearningRateScheduler callback and ModelCheckpoint callback.

In [ ]:
#Have fun ..... you just completed the project

The timeline for this is 2 weeks. In the last two weeks, we will be exploring further concepts in AIML or doing some fun activities in the field. Look forward to it :)